[![AnalyticsDojo](https://github.com/rpi-techfundamentals/fall2018-materials/blob/master/fig/final-logo.png?raw=1)](http://rpi.analyticsdojo.com)
<center><h1>Fastai - Revisiting Titanic</h1></center>
<center><h3><a href = 'http://rpi.analyticsdojo.com'>rpi.analyticsdojo.com</a></h3></center>


In [64]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html


In [0]:
from fastai import *
from fastai.tabular import * 
import numpy as np
import pandas as pd
import pandas as pd

train= pd.read_csv('https://raw.githubusercontent.com/rpi-techfundamentals/fall2018-materials/master/input/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/rpi-techfundamentals/fall2018-materials/master/input/test.csv')



In [66]:
#Create a categorical variable from the family count 
def family(x):
    if x < 2:
        return 'Single'
    elif x == 2:
        return 'Couple'
    elif x <= 4:
        return 'InterM'
    else:
        return 'Large'


for df in [train, test]:
    df['Title'] = df['Name'].str.split(',').str[1].str.split(' ').str[1]
    df['Title'] = df['Title'].replace(['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')
    df['Age']=df['Age'].fillna(df['Age'].median())
    df['Fare']=df['Fare'].fillna(df['Fare'].median())
    df['Embarked']=df['Embarked'].fillna('S')
    df['NameLength'] = df['Name'].map(lambda x: len(x))
    df['FamilyS'] = df['SibSp'] + df['Parch'] + 1
    df['FamilyS'] = df['FamilyS'].apply(family)
train.isnull().sum(axis = 0)


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Title            0
NameLength       0
FamilyS          0
dtype: int64

In [67]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,NameLength,FamilyS
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr.,23,Couple
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs.,51,Couple
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,22,Single
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs.,44,Couple
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr.,24,Single


In [0]:
dep_var = 'Survived'
cat_names = ['Pclass', 'Sex', 'Embarked', 'Title', 'FamilyS']
cont_names = ['Age', 'Fare', 'SibSp', 'Parch', 'NameLength']
procs = [FillMissing, Categorify, Normalize]
test_data = (TabularList.from_df(test, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs))



In [0]:

data = (TabularList.from_df(train, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(0,200)))
                           .label_from_df(cols=dep_var)
                           .add_test(test_data, label=0)
                           .databunch())

In [70]:
#Shows the Data
data.show_batch()

Pclass,Sex,Embarked,Title,FamilyS,Age,Fare,SibSp,Parch,NameLength,target
1,female,C,Miss.,Single,-0.1366,1.4643,-0.4610,-0.4658,-0.4163,1
3,female,S,Mrs.,Large,0.7213,-0.0433,0.5043,5.8949,3.2484,0
3,male,S,Mr.,Large,-0.1366,0.6815,7.2619,2.0785,-0.3085,0
1,male,S,Mr.,Couple,0.4873,0.8577,0.5043,-0.4658,0.2304,0
3,male,S,Mr.,Single,-0.3706,-0.5020,-0.4610,-0.4658,-0.5241,0


In [71]:
#Define our Learner
learn = tabular_learner(data, layers=[300,100], metrics=accuracy)
#fit the learner
learn.fit(7, 1e-2)  #Number of epocs and the learning rate. learn.save('final_train')

epoch,train_loss,valid_loss,accuracy
1,0.616099,0.580075,0.735000
2,0.523964,0.470934,0.775000
3,0.477717,0.444212,0.795000
4,0.445426,0.419657,0.830000
5,0.424187,0.438970,0.800000
6,0.411238,0.394977,0.855000
7,0.398757,0.422902,0.810000


In [72]:
#Show the results
learn.show_results()

Pclass,Sex,Embarked,Title,FamilyS,Age,Fare,SibSp,Parch,NameLength,target,prediction
3,male,S,Mr.,Couple,-0.6046,-0.4982,0.5043,-0.4658,-0.4163,0,0
1,female,C,Mrs.,Couple,0.6433,0.7144,0.5043,-0.4658,2.6017,1,1
3,female,S,Miss.,Single,-0.2926,-0.4855,-0.4610,-0.4658,-0.5241,1,0
1,female,S,Mrs.,Couple,0.4093,0.3700,0.5043,-0.4658,1.8472,1,1
3,male,S,Mr.,Single,0.4093,-0.4831,-0.4610,-0.4658,-0.3085,0,0


In [73]:
#This will get predictions
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = to_np(np.argmax(predictions, 1))
labels.shape




(418,)

In [0]:
#Writing to File
submission=pd.DataFrame(test.loc[:,['PassengerId']])
submission['Survived']=labels
#Any files you save will be available in the output tab below

submission.to_csv('submission.csv', index=False)

In [0]:
from google.colab import files
files.download('submission.csv')